In [1]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [5]:
import gzip
import simplejson as json

In [6]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [7]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [9]:

def describe(key):
    N = len(scripts)
    total = sum([script[key] for script in scripts])
    avg = total / N
    var = sum([(script[key] - avg)** 2 for script in scripts]) / N - 1
    s = var ** 0.5
    
    vals = sorted([script[key] for script in scripts])
    q25 = vals[N // 4]
    med = vals[N // 2]
    q75 = vals[-N // 4]
    


    return (total, avg, s, q25, med, q75)

In [10]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [11]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [12]:
bnf_names = {script['bnf_name'] for script in scripts}
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [13]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    bnf_name = script['bnf_name']
    groups[bnf_name].append(script)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [15]:
items_by_bnf_name = []

for bnf_name, scripts_ in groups.items():
    total = sum([script['items'] for script in scripts_])
    items_by_bnf_name.append((bnf_name, total))

In [19]:
items_by_bnf_name

[('MagnaPhate_Tab Chble 97.2mg', 41),
 ('Colecal_Tab 800u', 858),
 ('Dermalo_Bath Emollient', 87),
 ('Oxazepam_Tab 10mg', 283),
 ('Rewisca_Cap 150mg', 114),
 ('Lido_Oint 5%', 51),
 ('Sharpsafe 1.8L', 4),
 ('Cloral Betaine_Tab 707mg', 6),
 ('Flexi-Ban 10cm x 3.5m Sub-Compress Waddi', 3),
 ('Nicotine_Transdermal Patch 15mg/16hrs', 64),
 ('Oxycodone HCl/NaloxoneHCl_Tab 5/2.5mgM/R', 6),
 ('Mezolar Matrix_TransdermalPatch 75mcg/hr', 21),
 ('Micralax_Micro-Enem 5ml', 321),
 ('Morph Sulf_Tab 30mg M/R', 474),
 ('Dermacolor_Fixing Pdr', 13),
 ('Tegretol Prolonged Release_Tab 200mg M/R', 1767),
 ('Cod Liver Oil_Cap 400mg', 4),
 ('Adcal-D3_Tab Chble (Lem)', 3797),
 ('Slo-Phyllin_Cap 60mg', 22),
 ('Omnican Fine Needles Pen Inj Screw On 6m', 379),
 ('Unspecified Surg Adh Tape 5cm x 5m Perm', 2),
 ('Chlorhex Glucon_Emollient/Crm 1%', 2),
 ('Questran_Sach 9g (4g Of Ingredient)', 10),
 ('Lignospan Special_Inj 2% 2.2ml Cart', 26),
 ('Levocetirizine_Tab 5mg', 291),
 ('Betameth Val/Neomycin Sulf_Oint0.1/

In [23]:
max(items_by_bnf_name)

('palmdoc (Reagent)_Strips', 6)

In [24]:
def criterion(tup):
    return tup[1]

In [28]:
max(items_by_bnf_name, key=lambda x: x[1] )

('Omeprazole_Cap E/C 20mg', 113826)

In [29]:
max_item = [("Omeprazole_Cap E/C 20mg", 113826)]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [30]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [22]:
def group_by_field(data, fields):
    groups = {}
    for d in data:
        key = tuple(d[f] for f in fields)
        if key in groups:
            groups[key].append(d)
        else:
            groups[key] = [d]
    return groups

In [44]:
def sum_values(data, key):
    return sum([d[key] for d in data])

In [31]:
groups = group_by_field(scripts, ('bnf_name',))
test_max_item = ...

assert test_max_item == max_item

AssertionError: 

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [33]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = min(practice['post_code'], practice_postal[practice['code']])
    else:
        practice_postal[practice['code']] = practice['post_code']

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [34]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [37]:
joined = scripts[:]
for script in joined:
    practice_code = script['practice']
    script['post_code'] = practice_postal[practice_code]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [38]:
grouped = group_by_field(joined, ('post_code',))

In [44]:
postal_totals_all = [(post_code[0], sum_values(scripts_, 'items')) for post_code, scripts_ in grouped.items()]

In [45]:
postal_totals = sorted(postal_totals_all)[:100]

In [ ]:
items_by_post = ...

In [47]:
#postal_totals = [('B11 4BW', 20673)] * 100

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [56]:
#total_items_by_bnf_post = group_by_field(joined, ('post_code', 'bnf_name'))
grouped_by_bnf_code_and_post_code = group_by_field(joined, ('post_code', 'bnf_name'))
#assert len(total_items_by_bnf_post) == 141196

In [57]:
grouped_by_bnf_code_and_post_code[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF')]

[{'bnf_code': '0301011R0AAAPAP',
  'items': 300,
  'practice': 'M85078',
  'bnf_name': 'Salbutamol_Inha 100mcg (200 D) CFF',
  'nic': 622.5,
  'act_cost': 580.01,
  'quantity': 415,
  'post_code': 'B11 4BW'},
 {'bnf_code': '0301011R0AAAPAP',
  'items': 149,
  'practice': 'M85774',
  'bnf_name': 'Salbutamol_Inha 100mcg (200 D) CFF',
  'nic': 264.0,
  'act_cost': 246.25,
  'quantity': 176,
  'post_code': 'B11 4BW'},
 {'bnf_code': '0301011R0AAAPAP',
  'items': 257,
  'practice': 'Y02620',
  'bnf_name': 'Salbutamol_Inha 100mcg (200 D) CFF',
  'nic': 736.5,
  'act_cost': 685.0,
  'quantity': 491,
  'post_code': 'B11 4BW'}]

In [58]:
total_items_by_bnf_post = [(key, sum_values(val, 'items')) for key, val in grouped_by_bnf_code_and_post_code.items()]

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [59]:
def construct_dict(key, val):
    """ Accept a tuple and return a dictionary mapping the values of that tuple to a given array of keys"""
    return {'post_code':key[0], 'bnf_name':key[1], 'total':val}

total_items_by_bnf = [construct_dict(key, val) for key, val in total_items_by_bnf_post]

In [60]:
total_items_by_bnf[0]

{'post_code': 'SK11 6JL',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'total': 5}

In [68]:
grouped_by_post_code = group_by_field(total_items_by_bnf, ('post_code', ))
total_items_by_post = [(key[0], sum_values(val, 'total')) for key, val in grouped_by_post_code.items()]

In [69]:
total_items_by_post

[('SK11 6JL', 110071),
 ('CW5 5NX', 38797),
 ('CW1 3AW', 64104),
 ('CW7 1AT', 43164),
 ('CH65 6TG', 25090),
 ('CH1 4DS', 34915),
 ('TS10 4NW', 45161),
 ('TS24 7PW', 58207),
 ('TS17 0EE', 68388),
 ('TS23 2DG', 31646),
 ('TS3 6AL', 51402),
 ('TS1 2NX', 47623),
 ('OL4 1YN', 24687),
 ('OL1 1NL', 41046),
 ('BL9 0SN', 35275),
 ('WN7 1HR', 29076),
 ('BL3 5HP', 27147),
 ('BL1 8TU', 26132),
 ('M26 2SP', 37718),
 ('BL9 0NJ', 32062),
 ('M35 0AD', 37632),
 ('OL9 7AY', 28394),
 ('OL11 1DN', 21567),
 ('M30 0NU', 25597),
 ('M11 4EJ', 23166),
 ('SK6 1ND', 28313),
 ('WN3 5HL', 31149),
 ('WN2 5NG', 22795),
 ('WN7 2PE', 23130),
 ('BB2 1AX', 28254),
 ('BB3 1PY', 54514),
 ('FY2 0JG', 69118),
 ('FY4 1TJ', 62886),
 ('BB11 2DL', 34356),
 ('BB8 0JZ', 54380),
 ('BB9 7SR', 38224),
 ('BB7 2JG', 44585),
 ('BB4 5SL', 29388),
 ('LA1 1PN', 47335),
 ('FY5 2TZ', 44258),
 ('FY7 8GU', 34473),
 ('WA7 1AB', 41314),
 ('L36 7XY', 22965),
 ('L31 0DJ', 32065),
 ('WA10 2DJ', 40394),
 ('WA9 1LN', 29644),
 ('L7 6HD', 42569),
 ('L

In [63]:
assert len(total_items_by_bnf_post) == 141196

In [70]:
total_items = ...
assert len(total_items_by_post) == 118

In [76]:
sorted(grouped_by_post_code[('B11 4BW',)], key=lambda x: (x['total'], x['bnf_name']), reverse=True)[:5]

[{'post_code': 'B11 4BW',
  'bnf_name': 'Salbutamol_Inha 100mcg (200 D) CFF',
  'total': 706},
 {'post_code': 'B11 4BW', 'bnf_name': 'Paracet_Tab 500mg', 'total': 451},
 {'post_code': 'B11 4BW', 'bnf_name': 'Metformin HCl_Tab 500mg', 'total': 387},
 {'post_code': 'B11 4BW',
  'bnf_name': 'Lansoprazole_Cap 30mg (E/C Gran)',
  'total': 385},
 {'post_code': 'B11 4BW', 'bnf_name': 'Amoxicillin_Cap 500mg', 'total': 350}]

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [77]:
max_item_by_post = [sorted(val, key=lambda x: x['total'], reverse=True)[0] for val in grouped_by_post_code.values()]

In [80]:
max_item_by_post[:5]

[{'post_code': 'SK11 6JL',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 3219},
 {'post_code': 'CW5 5NX',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 1419},
 {'post_code': 'CW1 3AW',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 2364},
 {'post_code': 'CW7 1AT',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 1655},
 {'post_code': 'CH65 6TG',
  'bnf_name': 'Lansoprazole_Cap 30mg (E/C Gran)',
  'total': 688}]

In [82]:
total_items_by_post[:5]

[('SK11 6JL', 110071),
 ('CW5 5NX', 38797),
 ('CW1 3AW', 64104),
 ('CW7 1AT', 43164),
 ('CH65 6TG', 25090)]

In [90]:
post_code_target = 'CW7 1AT'

for post_code, total in total_items_by_post:
    if post_code == post_code_target:
        denom = total

In [89]:
def find_denom(post_code_target, total_items_by_post):
    for post_code, total in total_items_by_post:
        if post_code == post_code_target:
            return total
    

In [91]:
total_items_by_post_dict = dict(total_items_by_post)

In [93]:
%%timeit
total_items_by_post_dict['CW7 1AT']


46.9 ns ± 1.87 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [94]:
%%timeit
find_denom(post_code_target, total_items_by_post)

327 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [96]:
items_by_region_all = [(x['post_code'], x['bnf_name'], x['total'] / total_items_by_post_dict[x['post_code']]) for x in max_item_by_post]

In [97]:
items_by_region_all[:5]

[('SK11 6JL', 'Omeprazole_Cap E/C 20mg', 0.029244760200234393),
 ('CW5 5NX', 'Omeprazole_Cap E/C 20mg', 0.036574992911823076),
 ('CW1 3AW', 'Omeprazole_Cap E/C 20mg', 0.03687757394234369),
 ('CW7 1AT', 'Omeprazole_Cap E/C 20mg', 0.038342136965990176),
 ('CH65 6TG', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.027421283379832604)]

In [98]:
items_by_region = sorted(items_by_region_all)[:100]

In [99]:
items_by_region[:5]

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.02926161780958897),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03549462254750369),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.024095900880968663),
 ('B70 7AW', 'Paracet_Tab 500mg', 0.0266896608360023)]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [78]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [100]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*